In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize as spo
import statsmodels.api as sm
from statsmodels.graphics.dotplots import dot_plot
from scipy.interpolate import interp1d
import seaborn as sns
params = {'figure.figsize': (10,7),
          'axes.labelsize': 18,
          'font.size': 18,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'legend.fontsize': 16}
sns.set(palette='Set2', style='ticks', rc=params)

### Part 1 ###

In [2]:
import datetime
datetimeformat = '%Y-%m-%d %H:%M'

In [3]:
with open('inputs/day4.txt') as fp:
    lines = fp.readlines()
lines[0]

'[1518-06-03 00:32] wakes up\n'

In [4]:
def parse_line(line):
    first, middle, last = line.partition(']')
    when = datetime.datetime.strptime(first[1:], datetimeformat)
    event = last.strip()
    return when,event
parse_line(lines[0])

(datetime.datetime(1518, 6, 3, 0, 32), 'wakes up')

In [5]:
events = {}
for line in lines:
    when, event = parse_line(line)
    events[when] = event
timeline = sorted(events.keys())
print(timeline[:3])

[datetime.datetime(1518, 2, 3, 0, 3), datetime.datetime(1518, 2, 3, 0, 12), datetime.datetime(1518, 2, 3, 0, 28)]


In [6]:
timeline[0], timeline[-1]

(datetime.datetime(1518, 2, 3, 0, 3), datetime.datetime(1518, 11, 23, 0, 50))

In [7]:
set(events.values())

{'Guard #1039 begins shift',
 'Guard #1097 begins shift',
 'Guard #1249 begins shift',
 'Guard #1559 begins shift',
 'Guard #1601 begins shift',
 'Guard #2137 begins shift',
 'Guard #2381 begins shift',
 'Guard #2579 begins shift',
 'Guard #2879 begins shift',
 'Guard #2887 begins shift',
 'Guard #2969 begins shift',
 'Guard #3011 begins shift',
 'Guard #3083 begins shift',
 'Guard #3089 begins shift',
 'Guard #3119 begins shift',
 'Guard #3137 begins shift',
 'Guard #331 begins shift',
 'Guard #373 begins shift',
 'Guard #431 begins shift',
 'Guard #563 begins shift',
 'Guard #691 begins shift',
 'Guard #797 begins shift',
 'Guard #89 begins shift',
 'falls asleep',
 'wakes up'}

In [8]:
events[timeline[0]], events[timeline[-1]]

('Guard #691 begins shift', 'wakes up')

In [9]:
timeline[0], timeline[0].minute

(datetime.datetime(1518, 2, 3, 0, 3), 3)

In [10]:
import re
guard_re = re.compile(r'Guard #(\d+) begins')
import collections

In [11]:
sleeping = {}
for t in timeline:
    e = events[t]
    if "begin" in e:
        m = guard_re.match(e)
        guard = int(m.group(1))
    elif 'asleep' in e:
        asleep_start_min = t.minute
    elif 'wakes' in e:
        wake_min = t.minute
        sleeping_minutes = list(range(asleep_start_min, wake_min))
        c = sleeping.get(guard, collections.Counter())
        c.update(sleeping_minutes)
        sleeping[guard] = c
    else:
        raise ValueError(f'Mishandled event {e}')

In [12]:
total_sleeping = {guard: sum(c.values()) for guard,c in sleeping.items()}
max_sleeping = max(total_sleeping.values())
sleepiest_guard = [guard for guard in total_sleeping if total_sleeping[guard] == max_sleeping]
if len(sleepiest_guard) == 1:
    sleepiest_guard = sleepiest_guard[0]
else:
    raise ValueError(f'Multiple sleepiest guards: {sleepiest_guard}')
print(max_sleeping, sleepiest_guard)

sleepiest_minute = sleeping[sleepiest_guard].most_common(1)[0][0]
print(sleepiest_guard, sleepiest_minute, sleepiest_guard*sleepiest_minute)

538 2381
2381 44 104764


### Part 2 ###

In [13]:
most_asleep_by_min = {guard: sleeping[guard].most_common(1)[0] for guard in sleeping}

In [14]:
most_asleep_by_min

{691: (27, 6),
 2579: (37, 12),
 331: (25, 7),
 1097: (38, 14),
 797: (39, 9),
 3119: (29, 10),
 3083: (28, 12),
 3137: (41, 20),
 431: (26, 9),
 3011: (38, 6),
 2381: (44, 18),
 1039: (31, 7),
 373: (33, 12),
 1601: (47, 15),
 2969: (54, 8),
 2887: (32, 7),
 1249: (36, 13),
 89: (42, 11),
 3089: (41, 9),
 1559: (33, 7)}

In [27]:
def sorting_by_max_freq(guard):
    return most_asleep_by_min[guard][1]
guard_with_sleepiest_minute = sorted(most_asleep_by_min, key=sorting_by_max_freq, reverse=True)[0]
guards_sleepiest_minute = most_asleep_by_min[guard_with_sleepiest_minute][0]
print(guard_with_sleepiest_minute, guards_sleepiest_minute, 
     guard_with_sleepiest_minute*guards_sleepiest_minute)

3137 41 128617
